# Import data and clean

In [4]:
import json
import pandas as pd

In [5]:
# This data is received from Matt directly from the database. Will have to change the name
with open("data/2024-10-09_EVAPRD.EVA.json", 'r') as file:
        # normalize and get rid of first line
        data = json.load(file)
df = pd.json_normalize(data[1])
df

,OwnerName,Name,FolderName,CurrentHash,Baselines,IsArchive,_id.$oid,LastSaved.$date,RootTaskPackage.Code,RootTaskPackage._id,...,RootTaskPackage.PlannedEndDate,RootTaskPackage.ActualProgress,RootTaskPackage.PlannedProgress,RootTaskPackage.ForecastedProgress,RootTaskPackage.Baselines,RootTaskPackage.CostWeightAllocation,RootTaskPackage.DurationWeightAllocation,RootTaskPackage.ActualCost,RootTaskPackage.TaskIds,RootTaskPackage.TaskPackages
0,stefan@smaconsulting.ca,Lewis Farms Current,template,71F6980B4F682FAE3F0E398BEA40867C,"[{'BaselineNumber': 1, 'DateEffective': '2023-...",False,0a8ccf5d7013425d99ca2292,2024-10-07T20:24:58.216Z,CP-008169-01,649b4733510828c1260376fa,...,2028-12-31T07:00:00.000Z,[],[],[],[],50,50,82069131.07000000006,[],"[{'Code': '01', '_id': '649b4733510828c1260376..."


In [6]:
# narrow in on the taskpackages column
df = df["RootTaskPackage.TaskPackages"].loc[0]
df

[{'Code': '01',
  '_id': '649b4733510828c1260376fb',
  'DisplayIndex': '1',
  'Parent': '649b4733510828c1260376fa',
  'ParentDisplayIndex': '0',
  'Name': 'CONCEPT',
  'ForecastMethod': 'Default',
  'ConstantForcastValue': None,
  'PlannedCost': '2397887.82',
  'PlannedDuration': '1697',
  'PlannedStartDate': '2015-03-01T07:00:00.000Z',
  'PlannedEndDate': '2022-05-31T06:00:00.000Z',
  'ActualProgress': [],
  'PlannedProgress': [],
  'ForecastedProgress': [],
  'Baselines': [],
  'CostWeightAllocation': 'inherited',
  'DurationWeightAllocation': 'inherited',
  'ActualCost': '2397887.82',
  'TaskIds': [],
  'TaskPackages': [{'Code': '01-01',
    '_id': '649b4733510828c1260376fc',
    'DisplayIndex': '2',
    'Parent': '649b4733510828c1260376fb',
    'ParentDisplayIndex': '1',
    'Name': 'PROJECT TEAM',
    'ForecastMethod': 'Default',
    'ConstantForcastValue': None,
    'PlannedCost': '289648.18',
    'PlannedDuration': '1262',
    'PlannedStartDate': '2015-03-01T07:00:00.000Z',
    

# Recursively obtain the IDs of all tasks

In [7]:
# This function adds the list of ids from the key 'TaskIds' from the data into our temp variable 'taskIds'. Since the data is so heavily nested via the 'TaskPackages' key, we have to recursively iterate through the json via the 'TaskPackage' key.
def getTasks(df):
    # temp task list
    taskIds = []
    for i in range(len(df)):
        for key, val in df[i].items():
            if key == 'TaskIds':
                # accumulate tasks in the list
                taskIds += val
            elif key == 'TaskPackages':
                if val != None:
                    # recursively go through entire json
                    taskIds += getTasks(val)
    return taskIds

taskIds = getTasks(df)
taskIds

['649b4733510828c1260376fe',
 '649b4733510828c126037700',
 '649b4733510828c126037704',
 '649b4733510828c126037705',
 '649b4733510828c126037703',
 '649b4733510828c126037708',
 '649b4733510828c12603770b',
 '649b4733510828c12603770d',
 '649b4733510828c12603770e',
 '649b4733510828c12603770f',
 '649b4733510828c126037710',
 '649b4733510828c126037711',
 '649b4733510828c126037712',
 '649b4733510828c126037713',
 '649b4733510828c126037714',
 '649b4733510828c126037715',
 '649b4733510828c126037719',
 '649b4733510828c12603771a',
 '649b4733510828c12603771b',
 '649b4733510828c12603771c',
 '649b4733510828c12603771d',
 '649b4733510828c12603771f',
 '649b4733510828c126037720',
 '649b4733510828c126037723',
 '649b4733510828c126037724',
 '649b4733510828c126037725',
 '649b4733510828c126037726',
 '649b4733510828c126037727',
 '649b4733510828c12603772a',
 '649b4733510828c12603772d',
 '649b4733510828c126037730',
 '649b4733510828c126037731',
 '649b4733510828c126037732',
 '649b4733510828c126037734',
 '649b47335108

# Output as CSV

In [8]:
ids = pd.DataFrame(taskIds, columns=["ID"])
ids.to_csv('out.csv', index = False)